In [ ]:
using SpecialFunctions, StaticArrays, BasisFunctions, FrameFun, IntervalSets, DomainSets, LinearAlgebra, DomainIntegrals
using Plots
using CompactTranslatesDict, CardinalBSplines
using SimpleIntegralEquations


## Parameters of the problem

In [ ]:
T = Float64

wavenumber = T(5)

# Number of degrees of freedom in the basis
N = 128
# Number of collocation points (for collocation discretization)
M = N

# The degree of the B-splines we use to represent the solution
splinedegree = 1

In [ ]:
# We choose a kite-shaped domain but there are other options
obstacle = Kite{T}(2)
# obstacle = UnitCircle{T}()
# obstacle = Ellipse(zero(T), zero(T), T(3)/10, T(5)/10)

If the incoming wave is a plane wave, we can choose its direction and amplitude

In [ ]:
# We choose an incoming wave
direction = SVector(one(T), zero(T))
amplitude = one(T)

## Definition of the BEM matrices

In [ ]:
param = parameterization(obstacle)
paramdomain = domain(param)

SingleLayerPotential = Helmholtz_SLP_2D(wavenumber)
BIO = BoundaryIntegralOperator(SingleLayerPotential, obstacle, param)

# We use a basis of splines of linear degree in the parameter domain
basis = complex(BSplineTranslatesBasis(N, splinedegree, leftendpoint(paramdomain), rightendpoint(paramdomain)))
basis_obstacle = BasisFunctions.ParamDict(basis, param, obstacle)

# Collocation points in the parameter domain
coll_points = PeriodicEquispacedGrid(M, paramdomain)
coll_points_obstacle = mapped_grid(coll_points, param)

# Sampling operators for collocation and Galerkin
sampling_col = GridSampling(coll_points, Complex{T})
sampling_col_obstacle = GridSampling(coll_points_obstacle, Complex{T})
sampling_gal = ProjectionSampling(basis, measure(BIO))

# The boundary conditions (as functions)
bcond = make_parboundary_condition_planewave(param, wavenumber, direction, amplitude)
bcond_field = make_boundary_condition_planewave(wavenumber, direction, amplitude)

# And finally the BEM matrices (unassembled)
BEM_col = (sampling_col * BIO) * basis
BEM_gal = (sampling_gal * BIO) * basis

## BEM matrix assembly and solve

In [ ]:
# Two different quadrature strategies for the assembly of the BEM matrix
quad_qbf = QuadQBF{T}(splinedegree; oversamplingfactor=2);
quad_gk = QuadAdaptive{T}()

In [ ]:
@time compute_BEM_entry(BEM_col, 2, 2, quad_qbf)

In [ ]:
@time assemble!(BEM_col, quad_qbf);

In [ ]:
@time assemble!(BEM_gal, quad_qbf);

In [ ]:
A_col = copy(matrix(BEM_col))
b_col = sampling_col * bcond
coef_col = A_col \ b_col
density_col = Expansion(basis, coef_col)

In [ ]:
A_gal = copy(matrix(BEM_gal))
b_gal = sampling_gal * bcond
coef_gal = A_gal \ b_gal
density_gal = Expansion(basis, coef_gal)

## Evaluate the solution at a field

Since the boundary condition is a plane wave, and because we've used an integral equation of the first kind involving the singly layer potential, we know that the solution to the interior problem is precisely the plane wave.

In [ ]:
point = SVector(0.05, -0.2)
z_exact = bcond_field(point...)

In [ ]:
z_col = eval_field(BEM_col, density_col, point)
abs(z_col-z_exact) / abs(z_exact)

In [ ]:
z_gal = eval_field(BEM_gal, density_gal, point)
abs(z_gal - z_exact) / abs(z_exact)